<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We bring in libraries for handling data, pulling stock prices, and running a trading backtest. These tools give us everything we need to analyze the market and simulate a strategy.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from zipline.reloaded import run_algorithm
from zipline.api import order_target_percent, symbol, record, set_benchmark
from datetime import datetime

We set up our time range and define the list of stocks we want to analyze. We also start pulling close prices from Yahoo Finance for our selected stocks.

In [ ]:
start_date = pd.Timestamp('2018-01-01', tz='utc')
end_date = pd.Timestamp('2023-01-01', tz='utc')
symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'AMZN']
yf_data = yf.download(symbols, start=start_date, end=end_date, auto_adjust=True)['Close']
yf_data = yf_data.fillna(method='ffill')
yf_data = yf_data.dropna(how='all')
yf_data = yf_data[symbols]
yf_data.index = yf_data.index.tz_localize('utc')

This block creates a set of stock price data, fills in any missing values by carrying forward earlier prices, and narrows down our table so it only includes the stocks we care about. We also make sure that all the times are matched to the right timezone, which is required by our backtesting tool.

In [ ]:
panel = pd.Panel({sym: pd.DataFrame({'close': yf_data[sym]}) for sym in symbols})

We build an easy-to-use data structure from our cleaned price data so our backtesting engine gets the format it expects. Each stock's price history is stored in a simple table, so we avoid any messy data handling during the backtest.

## Strategy logic and execution

We write the setup and daily logic for our trading strategy. These functions tell the backtest how to start, which stocks to use, and what decisions to make each day.

In [ ]:
def initialize(context):
    context.symbols = [symbol(s) for s in symbols]
    set_benchmark(symbol('AAPL'))

In [ ]:
def handle_data(context, data):
    prices = np.array([data.current(s, 'price') for s in context.symbols])
    returns = np.array([data.history(s, 'price', 5, '1d').pct_change().mean() for s in context.symbols])
    mean = np.nanmean(returns)
    zscores = (returns - mean) / (np.nanstd(returns) + 1e-8)
    weights = -zscores
    weights = weights / np.nansum(np.abs(weights))
    for i, s in enumerate(context.symbols):
        order_target_percent(s, weights[i])
    record(leverage=np.sum(np.abs(weights)))

These functions create our rebalancing rules. The first part sets up our strategy at the start, picking the stocks and setting the benchmark we'll measure performance against. The daily function looks back at recent price changes, calculates a score for each stock, and decides how much to invest in each one. We use a simple process—buy underperformers and sell outperformers, based on how far their returns are from the average. Every day, we track how much we're investing to keep an eye on risk.

## Running the backtest

We launch the backtest using our prepared data and trading rules. Results are collected so we can measure performance and risk.

In [ ]:
metrics = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    capital_base=1e5,
    handle_data=handle_data,
    data=panel
)

The backtest runs the strategy over our chosen dates, using real stock prices. Each day, it follows the trading rules and tracks how our investment grows or shrinks. All the results are collected in one table for easy review.

## Viewing key results

We focus on two numbers—the value of our portfolio over time, and how much we're leveraging our bets.

In [ ]:
metrics[['portfolio_value', 'leverage']]

We pull just the core results so we can quickly see if our approach stays within reasonable risk levels and how our wealth changes day to day. It's a simple way to check if the strategy is holding up in practice.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.